# 🌿 Plant Disease Detection using Transfer Learning

This notebook demonstrates how to classify plant leaf diseases using a pre-trained model (MobileNetV2) and TensorFlow's high-level APIs. The dataset is downloaded from Kaggle and organized using `image_dataset_from_directory`.

In [ ]:
!pip install -q kaggle tensorflow opencv-python-headless matplotlib seaborn

## 🔑 Upload Kaggle API Key

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## 📦 Download PlantVillage Dataset

In [ ]:
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip -d plant_dataset

## 🗂️ Prepare Image Dataset

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

data_dir = "plant_dataset/PlantVillage"
batch_size = 32
img_height = 180
img_width = 180

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

class_names = train_ds.class_names
print("Class names:", class_names)


## 🤖 Build Model Using MobileNetV2

In [ ]:

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(input_shape=(180, 180, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


## 🏋️ Train the Model

In [ ]:

epochs = 5
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)


## 📈 Visualize Accuracy and Loss

In [ ]:

import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
